In [ ]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
fund_codes = ['MMM','ABT','ABBV']
start_date = '2019-11-30'
end_date = '2020-04-30'

In [ ]:
def loadHistoricalIndex(fund_codes, start_date, end_date):
    response_columns = fund_codes
    response_columns = ["date"] + fund_codes
    all_historical_prices = pd.read_csv("output.csv")
    subset_data = all_historical_prices[response_columns][
        all_historical_prices["date"] >= start_date
    ][all_historical_prices["date"] <= end_date]
    subset_data = subset_data.sort_values("date").reset_index(drop=True)
    for i in fund_codes:
        subset_data[f"{i}index"] = subset_data[i] / subset_data[i][0]
        subset_data = subset_data.fillna(0)
    subset_data = subset_data.fillna(0)
    subset_data = subset_data.drop(fund_codes, axis=1)
    subset_data.columns = response_columns
    return json.loads(subset_data.to_json(orient="records"))


In [ ]:
historical_index = loadHistoricalIndex(fund_codes, start_date, end_date)

In [ ]:
portfolio= [ {"fund": "MMM", "amount": 1000}, 
{"fund": "ABT", "amount": 1000},
{"fund":"ABBV", "amount":1000}]
strategy= {'rebalance': True, "rebalanceFrequency": "m"}

In [ ]:
rebalanceFrequency = strategy['rebalanceFrequency'].lower()

In [ ]:
num_funds = len(portfolio)

In [ ]:
fund_index = pd.DataFrame(historical_index)
fund_list = []
fund_amount = []
for i in portfolio:
    fund_list.append(i["fund"])
    fund_amount.append(i["amount"])
fund_percentage = [i /sum(fund_amount) for i in fund_amount]

In [ ]:
def backmodel (fund_amount, fund_codes, fund_index):
    fund_diag = np.zeros((num_funds,num_funds))
    np.fill_diagonal(fund_diag, fund_amount)
    return fund_index[fund_codes].dot(fund_diag)

In [ ]:
result_df = backmodel (fund_amount, fund_codes, fund_index)
result_df.columns = fund_codes
result_df['portfolio'] = result_df.sum(axis = 1)

In [ ]:
fund_codes = ['MMM','ABT','ABBV']
start_date = '2019-10-30'
end_date = '2020-04-30'

In [ ]:
frequency = {"y": 12, "q": 3, "m": 1}
start_period = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')
delta_months = relativedelta(end_date,start_period)

In [ ]:
diff_months = delta_months.months
if delta_months.days != 0:
    diff_months = diff_months + 1

In [ ]:
temp_amount = fund_amount
subset_list= []

In [ ]:

temp_amount = fund_amount
subset_list= []
for j in range(0, diff_months):
    end_period = start_period + relativedelta(months=frequency[rebalanceFrequency])
    start_period = start_period.strftime('%Y-%m-%d')
    end_period = end_period.strftime('%Y-%m-%d')
    subset_index = fund_index[fund_index['date'] >= start_period][fund_index['date'] <= end_period]
    temp_dates = subset_index['date']
    temp_df = backmodel(temp_amount, fund_codes, subset_index)
    temp_total = temp_df.iloc[(temp_df.shape[0] - 1)].sum()
    temp_amount = [temp_total * i for i in fund_percentage]
    temp_df['date'] = temp_dates
    subset_list.append(temp_df)
    start_period = datetime.strptime(end_period, '%Y-%m-%d')

In [ ]:
fund_index

In [ ]:
pd.concat(subset_list)

In [ ]:
fund_index["portfolio"] = fund_index[fund_list].sum(axis=1)
json.loads(fund_index[["date", "portfolio"]].to_json(orient="records"))
